In [1]:
print("Starting Python Notebook")

Starting Python Notebook


In [2]:
!brew install apache-spark

==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
To reinstall 3.5.0, run:
  brew reinstall apache-spark


In [3]:
print("Completed installing apache-spark")


Completed installing apache-spark


In [4]:
!brew install openjdk@11


To reinstall 11.0.20.1, run:
  brew reinstall openjdk@11


### The above Java installation doesn't work immediately. If you do a "re-install" it tells which commands need to be run to fix the path variable etc.

   ### (echo; echo 'eval "$(/opt/homebrew/bin/brew shellenv)"') >> /Users/ganapathychidambaram/.zprofile

   
   ### eval "$(/opt/homebrew/bin/brew shellenv)"

In [5]:
!which java


/usr/bin/java


In [6]:
!export JAVA_HOME=/usr
!export PATH=$JAVA_HOME/bin:$PATH

In [7]:
!java --version

openjdk 11.0.20.1 2023-08-24
OpenJDK Runtime Environment Homebrew (build 11.0.20.1+0)
OpenJDK 64-Bit Server VM Homebrew (build 11.0.20.1+0, mixed mode)


In [8]:
!export SPARK_HOME=/opt/homebrew/Cellar/apache-spark/3.3.0/libexec
!export PATH=/opt/homebrew/Cellar/apache-spark/3.3.0/bin:$PATH

# looks the case of the JAVA_HOME variable made all the difference    

!export PYSPARK_SUBMIT_ARGS="--master local[3] pyspark-shell"

In [9]:
!pip3 install pyspark
!pip3 install findspark

In [10]:
import findspark
findspark.init()

In [11]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

23/09/26 20:53:57 WARN Utils: Your hostname, Ganapathys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
23/09/26 20:53:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/26 20:53:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
applications_df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("/Users/ganapathychidambaram/Desktop/predixions/apps.tsv")

In [13]:
applications_df.show()

+------+--------+-----+--------------------+-------+
|UserID|WindowID|Split|     ApplicationDate|  JobID|
+------+--------+-----+--------------------+-------+
|    47|       1|Train|2012-04-04 15:56:...| 169528|
|    47|       1|Train|2012-04-06 01:03:...| 284009|
|    47|       1|Train|2012-04-05 02:40:...|   2121|
|    47|       1|Train|2012-04-05 02:37:...| 848187|
|    47|       1|Train|2012-04-05 22:44:...| 733748|
|    47|       1|Train|2012-04-05 02:34:...| 576958|
|    47|       1|Train|2012-04-05 22:55:...| 262470|
|    47|       1|Train|2012-04-05 02:38:...| 602298|
|    72|       1|Train|2012-04-02 22:36:...| 834662|
|    72|       1|Train|2012-04-07 15:19:...|1020903|
|    72|       1|Train|2012-04-07 17:38:...| 180313|
|    72|       1|Train|2012-04-30 20:05:...| 480634|
|    72|       1|Train|2012-04-20 02:51:...| 564184|
|    80|       1|Train|2012-04-04 10:53:...| 908909|
|   123|       1|Train|2012-04-02 20:57:...|1080147|
|   123|       1|Train|2012-04-02 21:03:...|10

In [14]:
applications_df.describe().show()

23/09/26 20:54:01 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+-------+--------------------+------------------+
|summary|            UserID|          WindowID|  Split|     ApplicationDate|             JobID|
+-------+------------------+------------------+-------+--------------------+------------------+
|  count|           1603111|           1603111|1603111|             1603111|           1603111|
|   mean| 762702.2528109407|3.6435674136101617|   null|                null| 565261.5271032386|
| stddev|422564.90726027224|2.0290820972908996|   null|                null|323913.61690280534|
|    min|           1000000|                 1|   Test|2012-04-01 00:00:...|                 1|
|    max|            999982|                 7|  Train|2012-06-26 23:59:...|            999999|
+-------+------------------+------------------+-------+--------------------+------------------+



pre-processing = Drop null values

In [15]:
applications_df = applications_df.dropna()

In [16]:
applications_df.describe().show()

+-------+------------------+------------------+-------+--------------------+------------------+
|summary|            UserID|          WindowID|  Split|     ApplicationDate|             JobID|
+-------+------------------+------------------+-------+--------------------+------------------+
|  count|           1603111|           1603111|1603111|             1603111|           1603111|
|   mean| 762702.2528109407|3.6435674136101617|   null|                null| 565261.5271032386|
| stddev|422564.90726027224|2.0290820972908996|   null|                null|323913.61690280534|
|    min|           1000000|                 1|   Test|2012-04-01 00:00:...|                 1|
|    max|            999982|                 7|  Train|2012-06-26 23:59:...|            999999|
+-------+------------------+------------------+-------+--------------------+------------------+



In [17]:
users_df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("/Users/ganapathychidambaram/Desktop/predixions/users.tsv")

In [18]:
users_df.show()

+------+--------+-----+------------+-----+-------+-------+-----------+--------------------+-------------------+----------------+--------------------+-----------------+-------------+--------------+
|UserID|WindowID|Split|        City|State|Country|ZipCode| DegreeType|               Major|     GraduationDate|WorkHistoryCount|TotalYearsExperience|CurrentlyEmployed|ManagedOthers|ManagedHowMany|
+------+--------+-----+------------+-----+-------+-------+-----------+--------------------+-------------------+----------------+--------------------+-----------------+-------------+--------------+
|    47|       1|Train|   Paramount|   CA|     US|  90723|High School|                null|1999-06-01 00:00:00|               3|                  10|              Yes|           No|             0|
|    72|       1|Train|     La Mesa|   CA|     US|  91941|   Master's|        Anthropology|2011-01-01 00:00:00|              10|                   8|              Yes|           No|             0|
|    80|       

In [19]:
unique_values = users_df.select('DegreeType').distinct()
unique_values.show()

+-----------+
| DegreeType|
+-----------+
|High School|
|       None|
| Vocational|
|        PhD|
|Associate's|
| Bachelor's|
|   Master's|
+-----------+



In [20]:
unique_values = users_df.select('Major').distinct()
print(unique_values.count())

47072


In [21]:
from pyspark.sql.functions import col, when
users_df.filter(col('Major') == "Not Applicable").count()

58760

In [22]:
users_df.filter(col('Major') == "None").count()

63

In [23]:
users_df.filter(col('Major').isNull()).count()

96897

In [24]:
users_df.filter(col('Major') == "null").count()

0

Pre-processing - convert None to Not Applicable

In [25]:
users_df.filter(col('CurrentlyEmployed').isNull()).count()

42076

In [26]:
# Replace null values with 'No' in a specific column (e.g., 'ColumnName')

users_df = users_df.na.fill('No', ['CurrentlyEmployed', 'ManagedOthers'])

In [27]:
unique_values = users_df.select('CurrentlyEmployed').distinct()
unique_values.show()

unique_values = users_df.select('ManagedOthers').distinct()
unique_values.show()


+-----------------+
|CurrentlyEmployed|
+-----------------+
|               No|
|              Yes|
+-----------------+

+-------------+
|ManagedOthers|
+-------------+
|           No|
|          Yes|
+-------------+



In [28]:
users_df = users_df.withColumn('Major', when(col('Major') == 'None', 'Not Applicable').otherwise(col('Major')))

In [29]:
users_df.count()

389708

In [30]:
!pip3 install numpy

In [31]:
from pyspark.ml.feature import StringIndexer

string_indexer = StringIndexer(inputCol='DegreeType', outputCol='DegreeType_encoded')
users_df_encoded = string_indexer.fit(users_df).transform(users_df)

users_df_encoded.show()

+------+--------+-----+------------+-----+-------+-------+-----------+--------------------+-------------------+----------------+--------------------+-----------------+-------------+--------------+------------------+
|UserID|WindowID|Split|        City|State|Country|ZipCode| DegreeType|               Major|     GraduationDate|WorkHistoryCount|TotalYearsExperience|CurrentlyEmployed|ManagedOthers|ManagedHowMany|DegreeType_encoded|
+------+--------+-----+------------+-----+-------+-------+-----------+--------------------+-------------------+----------------+--------------------+-----------------+-------------+--------------+------------------+
|    47|       1|Train|   Paramount|   CA|     US|  90723|High School|                null|1999-06-01 00:00:00|               3|                  10|              Yes|           No|             0|               2.0|
|    72|       1|Train|     La Mesa|   CA|     US|  91941|   Master's|        Anthropology|2011-01-01 00:00:00|              10|        

In [32]:

# Numerically encode these columns ['City', 'State', 'Country', 'CurrentlyEmployed', 'ManagedOthers']
categorical_columns = ['City', 'State', 'Country', 'CurrentlyEmployed', 'ManagedOthers']

for col in categorical_columns:
    encoded_column = col + "_encoded"
    string_indexer = StringIndexer(inputCol=col, outputCol=encoded_column)
    users_df_encoded = string_indexer.fit(users_df_encoded).transform(users_df_encoded)

# Show the resulting DataFrame with the encoded column
users_df_encoded.show()

+------+--------+-----+------------+-----+-------+-------+-----------+--------------------+-------------------+----------------+--------------------+-----------------+-------------+--------------+------------------+------------+-------------+---------------+-------------------------+---------------------+
|UserID|WindowID|Split|        City|State|Country|ZipCode| DegreeType|               Major|     GraduationDate|WorkHistoryCount|TotalYearsExperience|CurrentlyEmployed|ManagedOthers|ManagedHowMany|DegreeType_encoded|City_encoded|State_encoded|Country_encoded|CurrentlyEmployed_encoded|ManagedOthers_encoded|
+------+--------+-----+------------+-----+-------+-------+-----------+--------------------+-------------------+----------------+--------------------+-----------------+-------------+--------------+------------------+------------+-------------+---------------+-------------------------+---------------------+
|    47|       1|Train|   Paramount|   CA|     US|  90723|High School|         

In [33]:
columns_to_drop = ['City', 'State', 'Country', 'CurrentlyEmployed', 'ManagedOthers', 'DegreeType']
users_df_encoded = users_df_encoded.drop(*columns_to_drop)
users_df_encoded.show()

+------+--------+-----+-------+--------------------+-------------------+----------------+--------------------+--------------+------------------+------------+-------------+---------------+-------------------------+---------------------+
|UserID|WindowID|Split|ZipCode|               Major|     GraduationDate|WorkHistoryCount|TotalYearsExperience|ManagedHowMany|DegreeType_encoded|City_encoded|State_encoded|Country_encoded|CurrentlyEmployed_encoded|ManagedOthers_encoded|
+------+--------+-----+-------+--------------------+-------------------+----------------+--------------------+--------------+------------------+------------+-------------+---------------+-------------------------+---------------------+
|    47|       1|Train|  90723|                null|1999-06-01 00:00:00|               3|                  10|             0|               2.0|       924.0|          2.0|            0.0|                      0.0|                  0.0|
|    72|       1|Train|  91941|        Anthropology|2011

In [34]:
from pyspark.sql.functions import count, lit

# Calculate the frequency of each unique value in the 'Major' column
major_frequencies = users_df_encoded.groupBy('Major').agg(count(lit(1)).alias('major_frequency'))

# Join the frequency information back to the original DataFrame
users_df_encoded = users_df_encoded.join(major_frequencies, on='Major', how='left')

# Replace null values in the 'Frequency' column with 0
users_df_encoded = users_df_encoded.na.fill(0, ['major_frequency'])

users_df_encoded = users_df_encoded.drop('Major')
users_df_encoded.show()

+------+--------+-----+-------+-------------------+----------------+--------------------+--------------+------------------+------------+-------------+---------------+-------------------------+---------------------+---------------+
|UserID|WindowID|Split|ZipCode|     GraduationDate|WorkHistoryCount|TotalYearsExperience|ManagedHowMany|DegreeType_encoded|City_encoded|State_encoded|Country_encoded|CurrentlyEmployed_encoded|ManagedOthers_encoded|major_frequency|
+------+--------+-----+-------+-------------------+----------------+--------------------+--------------+------------------+------------+-------------+---------------+-------------------------+---------------------+---------------+
|    47|       1|Train|  90723|1999-06-01 00:00:00|               3|                  10|             0|               2.0|       924.0|          2.0|            0.0|                      0.0|                  0.0|              0|
|    72|       1|Train|  91941|2011-01-01 00:00:00|              10|        

In [35]:
user_history_df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("/Users/ganapathychidambaram/Desktop/predixions/user_history.tsv")

In [36]:
user_history_df.show()


+------+--------+-----+--------+--------------------+
|UserID|WindowID|Split|Sequence|            JobTitle|
+------+--------+-----+--------+--------------------+
|    47|       1|Train|       1|National Space Co...|
|    47|       1|Train|       2|   Detention Officer|
|    47|       1|Train|       3|Passenger Screene...|
|    72|       1|Train|       1|Lecturer, Departm...|
|    72|       1|Train|       2|   Student Assistant|
|    72|       1|Train|       3|   Elderly Caregiver|
|    72|       1|Train|       4|                null|
|    72|       1|Train|       5|Department Assist...|
|    72|       1|Train|       6|Book Reviews Edit...|
|    72|       1|Train|       7|  Graduate Assistant|
|    72|       1|Train|       8|                null|
|    72|       1|Train|       9|  Research Assistant|
|    72|       1|Train|      10|Docent, C.E. Smit...|
|    80|       1|Train|       1|Auto Publishing/E...|
|    80|       1|Train|       2|Enhanced Baker Ce...|
|    80|       1|Train|     

In [37]:
unique_values = user_history_df.select('JobTitle').distinct()
unique_values.count()

657162

In [38]:
user_history_df.count()

1753901

In [39]:
job_title_frequencies = user_history_df.groupBy('JobTitle').agg(count(lit(1)).alias('job_title_freq'))
job_title_frequencies.show()

+--------------------+--------------+
|            JobTitle|job_title_freq|
+--------------------+--------------+
|Commercial and re...|             1|
|      Service Writer|           109|
|Relief Production...|             1|
|Lead Field Techni...|             1|
|Pa. - Business Un...|             1|
|Field - Service E...|             1|
|Journeyman Electr...|           447|
|          Enumerator|           240|
|Health Service Sp...|             1|
|CUSTODIAN OF MEDI...|             1|
|   Procurement Buyer|             3|
|         Bank Teller|          1081|
|Customer Service ...|             1|
|Sr. Staffing & St...|             1|
|Minster Press Ope...|             1|
|Country General M...|             1|
|   Compliance Intern|             5|
|Account Services ...|             8|
|Paralegal Supervisor|             6|
|Commercial Loan C...|             1|
+--------------------+--------------+
only showing top 20 rows



In [40]:
# Join the frequency information back to the original DataFrame
user_history_df = user_history_df.join(job_title_frequencies, on='JobTitle', how='left')

# Replace null values in the 'Frequency' column with 0
user_history_df = user_history_df.na.fill(0, ['job_title_freq'])

# Drop the original 'JobTitle' column
user_history_df = user_history_df.drop('JobTitle')

# Show the resulting DataFrame
user_history_df.show()

+-------+--------+-----+--------+--------------+
| UserID|WindowID|Split|Sequence|job_title_freq|
+-------+--------+-----+--------+--------------+
| 393222|       4|Train|       3|             1|
| 393090|       4|Train|       4|           249|
| 393090|       4|Train|       5|           249|
| 586219|       7|Train|       1|             1|
|1034616|       1|Train|       1|            15|
|1034634|       1|Train|       3|             1|
| 393090|       4|Train|       6|             1|
| 337715|       5|Train|       5|            15|
|     80|       1|Train|       3|             1|
| 586173|       7|Train|       7|           132|
| 515554|       3|Train|       2|             1|
| 709314|       2|Train|       2|             1|
| 586173|       7|Train|       5|            41|
|1034664|       1|Train|       3|         15645|
| 395265|       6|Train|       5|         15645|
| 586221|       7|Train|       2|         15645|
| 337728|       5|Train|       3|             1|
| 709176|       2|Tr

In [41]:
from pyspark.sql.functions import collect_list, first


compressed_user_hist_df = user_history_df.groupBy('UserID').agg(
    collect_list('Sequence').alias('Sequence'),
    collect_list('job_title_freq').alias('job_title_freq'),
    first('WindowID').alias('WindowID'),
    first('Split').alias('Split')
)

In [42]:
compressed_user_hist_df.show()

+-------+--------------------+--------------------+--------+-----+
| UserID|            Sequence|      job_title_freq|WindowID|Split|
+-------+--------------------+--------------------+--------+-----+
|1000015|[2, 5, 3, 7, 6, 1...|[913, 634, 1, 15,...|       1| Test|
|1000070|        [4, 3, 2, 1]|[8577, 161, 41, 278]|       7|Train|
|1000127|        [4, 1, 2, 3]|  [1513, 47, 47, 47]|       7|Train|
|1000128|[1, 2, 6, 4, 3, 5...|[164, 11, 159, 1,...|       7|Train|
|1000131|[4, 1, 6, 3, 5, 2...|[13, 238, 1, 274,...|       5|Train|
| 100014|     [1, 3, 4, 5, 2]| [1813, 0, 0, 0, 36]|       4|Train|
|1000205|                 [1]|                 [1]|       5|Train|
|1000272|        [4, 1, 3, 2]|  [2779, 8577, 1, 1]|       3|Train|
|1000284|     [5, 4, 2, 1, 3]|[97, 1, 181, 1622...|       7|Train|
|1000316|[7, 1, 6, 2, 3, 5...|[218, 2, 2, 1, 1,...|       1|Train|
|1000353|           [3, 1, 2]|         [272, 1, 1]|       4|Train|
|1000401|                 [1]|                 [1]|       2|Tr

In [43]:
jobs_df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("/Users/ganapathychidambaram/Desktop/predixions/jobs.tsv")

In [44]:
jobs_df.show()

+-----+--------+--------------------+--------------------+--------------------+-----------------+-----+-------+-----+--------------------+-------------------+
|JobID|WindowID|               Title|         Description|        Requirements|             City|State|Country| Zip5|           StartDate|            EndDate|
+-----+--------+--------------------+--------------------+--------------------+-----------------+-----+-------+-----+--------------------+-------------------+
|    1|       1|Security Engineer...|<p>Security Clear...|<p>SKILL SET</p>\...|       Washington|   DC|     US|20531|2012-03-07 13:17:...|2012-04-06 23:59:59|
|    4|       1|SAP Business Anal...|<strong>NO Corp. ...|<p><b>WHAT YOU NE...|        Charlotte|   NC|     US|28217|2012-03-21 02:03:...|2012-04-20 23:59:59|
|    7|       1|P/T HUMAN RESOURC...|<b>    <b> P/T HU...|Please refer to t...|      Winter Park|   FL|     US|32792|2012-03-02 16:36:...|2012-04-01 23:59:59|
|    8|       1|Route Delivery Dr...|CITY BEVE

In [45]:
# Replace null values in 'Title' column with an empty string
jobs_df = jobs_df.withColumn("Title", when(jobs_df["Title"].isNull(), "").otherwise(jobs_df["Title"]))

# Replace null values in 'Description' and 'Requirements' columns
jobs_df = jobs_df.fillna({"Description": "<p>No description available</p>", "Requirements": "<p>No requirements available</p>"})

# Show the resulting DataFrame
jobs_df.show()

+-----+--------+--------------------+--------------------+--------------------+-----------------+-----+-------+-----+--------------------+-------------------+
|JobID|WindowID|               Title|         Description|        Requirements|             City|State|Country| Zip5|           StartDate|            EndDate|
+-----+--------+--------------------+--------------------+--------------------+-----------------+-----+-------+-----+--------------------+-------------------+
|    1|       1|Security Engineer...|<p>Security Clear...|<p>SKILL SET</p>\...|       Washington|   DC|     US|20531|2012-03-07 13:17:...|2012-04-06 23:59:59|
|    4|       1|SAP Business Anal...|<strong>NO Corp. ...|<p><b>WHAT YOU NE...|        Charlotte|   NC|     US|28217|2012-03-21 02:03:...|2012-04-20 23:59:59|
|    7|       1|P/T HUMAN RESOURC...|<b>    <b> P/T HU...|Please refer to t...|      Winter Park|   FL|     US|32792|2012-03-02 16:36:...|2012-04-01 23:59:59|
|    8|       1|Route Delivery Dr...|CITY BEVE

In [46]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.sql.functions import col, concat, lit

# Concatenate the text from different columns into a single column
jobs_df = jobs_df.withColumn("combined_text", concat(col("Description"), lit(" "), col("Requirements")))

# Tokenize the combined text
tokenizer = Tokenizer(inputCol="combined_text", outputCol="tokens")
jobs_df = tokenizer.transform(jobs_df)

# Remove stopwords
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
jobs_df = stopwords_remover.transform(jobs_df)

# Compute Term Frequencies
count_vectorizer = CountVectorizer(inputCol="filtered_tokens", outputCol="tf_features")
count_vectorizer_model = count_vectorizer.fit(jobs_df)
jobs_df = count_vectorizer_model.transform(jobs_df)

# Compute Inverse Document Frequencies
idf = IDF(inputCol="tf_features", outputCol="tfidf_features")
idf_model = idf.fit(jobs_df)
jobs_df = idf_model.transform(jobs_df)

# Drop intermediate columns
jobs_df = jobs_df.drop("combined_text", "tokens", "filtered_tokens", "tf_features")

# Drop the source columns
jobs_df = jobs_df.drop("Title", "Description", "Requirements")


# Show the resulting DataFrame
jobs_df.show()


23/09/26 20:55:41 WARN MemoryStore: Not enough space to cache rdd_279_1 in memory! (computed 18.9 MiB so far)
23/09/26 20:55:41 WARN MemoryStore: Not enough space to cache rdd_279_6 in memory! (computed 15.0 MiB so far)
23/09/26 20:55:41 WARN BlockManager: Persisting block rdd_279_1 to disk instead.
23/09/26 20:55:41 WARN BlockManager: Persisting block rdd_279_6 to disk instead.
23/09/26 20:55:41 WARN MemoryStore: Not enough space to cache rdd_279_0 in memory! (computed 14.5 MiB so far)
23/09/26 20:55:41 WARN BlockManager: Persisting block rdd_279_0 to disk instead.
23/09/26 20:55:41 WARN MemoryStore: Not enough space to cache rdd_279_4 in memory! (computed 19.3 MiB so far)
23/09/26 20:55:41 WARN BlockManager: Persisting block rdd_279_4 to disk instead.
23/09/26 20:55:41 WARN MemoryStore: Not enough space to cache rdd_279_2 in memory! (computed 19.2 MiB so far)
23/09/26 20:55:41 WARN BlockManager: Persisting block rdd_279_2 to disk instead.
23/09/26 20:55:41 WARN MemoryStore: Not enoug

+-----+--------+-----------------+-----+-------+-----+--------------------+-------------------+--------------------+
|JobID|WindowID|             City|State|Country| Zip5|           StartDate|            EndDate|      tfidf_features|
+-----+--------+-----------------+-----+-------+-----+--------------------+-------------------+--------------------+
|    1|       1|       Washington|   DC|     US|20531|2012-03-07 13:17:...|2012-04-06 23:59:59|(262144,[0,5,7,10...|
|    4|       1|        Charlotte|   NC|     US|28217|2012-03-21 02:03:...|2012-04-20 23:59:59|(262144,[1,2,5,8,...|
|    7|       1|      Winter Park|   FL|     US|32792|2012-03-02 16:36:...|2012-04-01 23:59:59|(262144,[0,2,4,10...|
|    8|       1|          Orlando|   FL|     US| null|2012-03-03 09:01:...|2012-04-02 23:59:59|(262144,[1,18,22,...|
|    9|       1|          Orlando|   FL|     US| null|2012-03-03 09:01:...|2012-04-02 23:59:59|(262144,[0,7,18,2...|
|   10|       1|     Ormond Beach|   FL|     US|32174|2012-03-05

23/09/26 20:56:55 WARN DAGScheduler: Broadcasting large task binary with size 8.9 MiB


In [47]:
# Numerically encode these columns ['City', 'State', 'Country', 'CurrentlyEmployed', 'ManagedOthers']
categorical_columns = ['City', 'State', 'Country']

for col in categorical_columns:
    encoded_column = col + "_encoded"
    string_indexer = StringIndexer(inputCol=col, outputCol=encoded_column)
    jobs_df = string_indexer.fit(jobs_df).transform(jobs_df)

columns_to_drop = ['City', 'State', 'Country']
jobs_df = jobs_df.drop(*columns_to_drop)
# Show the resulting DataFrame with the encoded column
jobs_df.show()

+-----+--------+-----+--------------------+-------------------+--------------------+------------+-------------+---------------+
|JobID|WindowID| Zip5|           StartDate|            EndDate|      tfidf_features|City_encoded|State_encoded|Country_encoded|
+-----+--------+-----+--------------------+-------------------+--------------------+------------+-------------+---------------+
|    1|       1|20531|2012-03-07 13:17:...|2012-04-06 23:59:59|(262144,[0,5,7,10...|        12.0|         32.0|            0.0|
|    4|       1|28217|2012-03-21 02:03:...|2012-04-20 23:59:59|(262144,[1,2,5,8,...|         6.0|          8.0|            0.0|
|    7|       1|32792|2012-03-02 16:36:...|2012-04-01 23:59:59|(262144,[0,2,4,10...|       480.0|          2.0|            0.0|
|    8|       1| null|2012-03-03 09:01:...|2012-04-02 23:59:59|(262144,[1,18,22,...|        17.0|          2.0|            0.0|
|    9|       1| null|2012-03-03 09:01:...|2012-04-02 23:59:59|(262144,[0,7,18,2...|        17.0|       

23/09/26 20:57:06 WARN DAGScheduler: Broadcasting large task binary with size 9.3 MiB
